# SerpAPI Tool Helper

This notebook provides a quick rundown of how we construct an async SerpAPI tool. Let's start by query SerpAPI synchronously via their Python SDK:

In [2]:
from serpapi import GoogleSearch
from getpass import getpass

SERPAPI_API_KEY = getpass("Enter your SerpAPI API key: ")

params = {
    "api_key": SERPAPI_API_KEY,
    "engine": "google",
    "q": "latest news in the world",
}

search = GoogleSearch(params)
results = search.get_dict()

print(results)

{'search_metadata': {'id': '67ed8cd2caa734ab65e16a39', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/6abcbb2cff4fdf59/67ed8cd2caa734ab65e16a39.json', 'created_at': '2025-04-02 19:15:30 UTC', 'processed_at': '2025-04-02 19:15:30 UTC', 'google_url': 'https://www.google.com/search?q=latest+news+in+the+world&oq=latest+news+in+the+world&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/6abcbb2cff4fdf59/67ed8cd2caa734ab65e16a39.html', 'total_time_taken': 0.48}, 'search_parameters': {'engine': 'google', 'q': 'latest news in the world', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': 'latest news in the world', 'total_results': 1940000000, 'time_taken_displayed': 0.3, 'organic_results_state': 'Results for exact spelling'}, 'knowledge_graph': {'entity_type': 'related_questions'}, 'inline_videos': [{'position': 1, 'title': 'LIVE | Putin’s UNPRECEDENTED Speech Shakes The World ...', 'link': 'https://www.y

Our results are provided in the `"organic_results"` key:

In [3]:
results["organic_results"]

[{'position': 1,
  'title': 'World news - breaking news, video, headlines and opinion',
  'link': 'https://www.cnn.com/world',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.cnn.com/world&ved=2ahUKEwjrm63Qh7qMAxVjF1kFHUSAM7kQFnoECDIQAQ',
  'displayed_link': 'https://www.cnn.com › world',
  'favicon': 'https://serpapi.com/searches/67ed8cd2caa734ab65e16a39/images/a1e5f2d48cb5e598291b88bdfff01798cff16e6d023b0c62d12765cd24da488e.png',
  'snippet': "World · Two arrested as investigation into 'QatarGate' in Israel deepens · Trump accuses Ukraine's Zelensky of 'trying to back out' of proposed minerals deal.",
  'snippet_highlighted_words': ["Two arrested as investigation into 'QatarGate' in Israel deepens"],
  'sitelinks': {'inline': [{'title': 'Impact Your World',
     'link': 'https://edition.cnn.com/world/impact-your-world'},
    {'title': 'Sony World Photography...',
     'link': 'https://edition.cnn.com/2025/02/25/style/sony-world-photog

We reformat this to extract only the most relevant information, such as the the title, source, link, and snippet. Let's use pydantic `BaseModel` to define this structure.

In [4]:
from pydantic import BaseModel

class Article(BaseModel):
    title: str
    source: str
    link: str
    snippet: str

    @classmethod
    def from_serpapi_result(cls, result: dict) -> "Article":
        return cls(
            title=result["title"],
            source=result["source"],
            link=result["link"],
            snippet=result["snippet"],
        )

In [5]:
articles = [Article.from_serpapi_result(result) for result in results["organic_results"]]

## Using Async

All of this works, but it unfortunately is not async and the SerpAPI SDK does not support async either, so we much query the API directly using the `aiohttp` library.

In [6]:
import aiohttp

async with aiohttp.ClientSession() as session:
    async with session.get(
        "https://serpapi.com/search",
        params=params
    ) as response:
        results = await response.json()

results["organic_results"]

[{'position': 1,
  'title': 'World news - breaking news, video, headlines and opinion',
  'link': 'https://www.cnn.com/world',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.cnn.com/world&ved=2ahUKEwjrm63Qh7qMAxVjF1kFHUSAM7kQFnoECDIQAQ',
  'displayed_link': 'https://www.cnn.com › world',
  'favicon': 'https://serpapi.com/searches/67ed8cd2caa734ab65e16a39/images/a1e5f2d48cb5e598291b88bdfff01798cff16e6d023b0c62d12765cd24da488e.png',
  'snippet': "World · Two arrested as investigation into 'QatarGate' in Israel deepens · Trump accuses Ukraine's Zelensky of 'trying to back out' of proposed minerals deal.",
  'snippet_highlighted_words': ["Two arrested as investigation into 'QatarGate' in Israel deepens"],
  'sitelinks': {'inline': [{'title': 'Impact Your World',
     'link': 'https://edition.cnn.com/world/impact-your-world'},
    {'title': 'Sony World Photography...',
     'link': 'https://edition.cnn.com/2025/02/25/style/sony-world-photog

With that, we have all we need to build a fully async serpapi tool.

In [7]:
from langchain_core.tools import tool

@tool
async def serpapi(query: str) -> list[Article]:
    """Use this tool to search the web."""
    params = {
        "api_key": SERPAPI_API_KEY,
        "engine": "google",
        "q": query,
    }
    async with aiohttp.ClientSession() as session:
        async with session.get(
            "https://serpapi.com/search",
            params=params
        ) as response:
            results = await response.json()
    return [Article.from_serpapi_result(result) for result in results["organic_results"]]

Note that because this tool is async, we cannot use `tool.func` to call it as before:

In [14]:
serpapi.func

Instead, we use `tool.coroutine`:

In [12]:
serpapi.coroutine

<function __main__.serpapi(query: str) -> list[__main__.Article]>

---